In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from nuztf.neutrino_scanner import NeutrinoScanner
from astropy import units as u
from nuwinter.avro import load_avro
from nuwinter.plot import ann_fields, generate_single_page
from nuwinter.utils import deduplicate_df
from nuwinter.paths import get_pdf_path
from pathlib import Path
import pandas as pd
import json
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib.colors import Normalize
from matplotlib.ticker import MultipleLocator
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from tqdm import tqdm
from nuztf.skymap_scanner import SkymapScanner
from winterapi import WinterAPI
from astropy.time import Time
from mocpy import MOC
from astropy.coordinates import (
    ICRS,
    Angle,
    BaseCoordinateFrame,
    Galactic,
    Latitude,
    Longitude,
    SkyCoord,
)
from astropy.io import fits
import healpy as hp
import logging







/Users/robertstein/anaconda3/envs/nuwinter_env/lib/python3.11/site-packages/ligo/lw/lsctables.py:89: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
# Program details
program = "2023A001" # WINTER EMGW

name = "S250206dm"

prob_threshold = 0.95
n_days = 14

image_type="diff"

In [4]:
winter = WinterAPI()
program_list = winter.get_programs()
print(f"Available programs: {program_list}")
assert program in program_list, f"program {program} not found! Add this program first."

INFO:winterapi.messenger:API ping success is True
INFO:winterapi.messenger:Server requires minimum winterapi version: 1.5.0
INFO:winterapi.messenger:Local winterapi version: 1.3.0


Available programs: ['2023A000', '2023A001', '2023A002', '2023A004', '2023A010', '2023A012', '2023A999', '2024A000', '2024A001', '2024A002', '2024A006']


In [5]:
nu = NeutrinoScanner(name) if "IC" in name else SkymapScanner(
    event=name,
    prob_threshold=prob_threshold,
    n_days=n_days,
    output_nside=1024
)

INFO:nuztf.skymap:Obtaining skymap from GraceDB
INFO:nuztf.skymap:Found voevent S250206dm-7-Update.xml
INFO:nuztf.skymap:Latest skymap URL: https://gracedb.ligo.org/api/superevents/S250206dm/files/Bilby.offline1.multiorder.fits,0
INFO:nuztf.skymap:Saving to: /Users/robertstein/Data/nuztf/skymaps/S250206dm_rev7_Bilby.offline1.multiorder.fits,0
INFO:nuztf.skymap:Reading file: /Users/robertstein/Data/nuztf/skymaps/S250206dm_rev7_Bilby.offline1.multiorder.fits,0
INFO:nuztf.skymap:Rasterising skymap to convert to nested format
INFO:nuztf.skymap:Summed probability is 100.0%
INFO:nuztf.skymap:Regridding to nside 1024
INFO:nuztf.skymap:Event time: 2025-02-06T21:25:30.452
INFO:nuztf.skymap:Reading map
INFO:nuztf.skymap:Threshold found! 
 To reach 95.00% of probability, pixels with probability greater than 3.9e-07 are included.
INFO:nuztf.skymap_scanner:hottest_pixel: 38.17319848293299 53.23540420437919
INFO:nuztf.skymap_scanner:Checking which pixels are within the contour:
100%|████████████████

In [6]:
nu.t_min.isot.split("T")[0].replace("-", "")

'20250206'

In [7]:
t_end = min([nu.default_t_max, Time.now()])

res, images = winter.query_images_by_program(
    program, image_type=image_type,
    start_date=nu.t_min.isot.split("T")[0].replace("-", ""),
    end_date=t_end.isot.split("T")[0].replace("-", "")
)

Querying images for 2023A001 between 20250206 and 20250209 of type 'diff'


In [8]:
res.json()["msg"]

'Request passed validation! Found 86 images.'

In [9]:
images

,progname,nightdate,targname,ra,dec,fid,utctime,fieldid,image_type,savepath,lastmodified,pipeversion
0,2023A001,2025-02-08,S250206dm,36.099724,55.324870,2,2025-02-08T15:54:14.173000+00:00,3635,diff,/data/loki/data/winter/20250207/diffs/WINTERca...,2025-02-08T11:08:34.697189+00:00,0.19.0
1,2023A001,2025-02-08,S250206dm,36.106360,54.977524,2,2025-02-08T15:54:14.173000+00:00,3635,diff,/data/loki/data/winter/20250207/diffs/WINTERca...,2025-02-08T11:08:34.697189+00:00,0.19.0
2,2023A001,2025-02-08,S250206dm,36.124516,54.640247,2,2025-02-08T15:54:14.173000+00:00,3635,diff,/data/loki/data/winter/20250207/diffs/WINTERca...,2025-02-08T11:08:34.697189+00:00,0.19.0
3,2023A001,2025-02-08,S250206dm,37.156340,54.643085,2,2025-02-08T15:54:14.173000+00:00,3635,diff,/data/loki/data/winter/20250207/diffs/WINTERca...,2025-02-08T11:08:34.697189+00:00,0.19.0
4,2023A001,2025-02-08,S250206dm,37.164936,55.328064,2,2025-02-08T15:54:14.173000+00:00,3635,diff,/data/loki/data/winter/20250207/diffs/WINTERca...,2025-02-08T11:08:34.697189+00:00,0.19.0
...,...,...,...,...,...,...,...,...,...,...,...,...
81,2023A001,2025-02-08,S250206dm,33.658140,51.991096,2,2025-02-08T15:02:14.115000+00:00,4268,diff,/data/loki/data/winter/20250207/diffs/WINTERca...,2025-02-08T11:08:32.515195+00:00,0.19.0
82,2023A001,2025-02-08,S250206dm,33.675022,51.653760,2,2025-02-08T15:02:14.115000+00:00,4268,diff,/data/loki/data/winter/20250207/diffs/WINTERca...,2025-02-08T11:08:32.515195+00:00,0.19.0
83,2023A001,2025-02-08,S250206dm,34.636536,51.656536,2,2025-02-08T15:02:14.115000+00:00,4268,diff,/data/loki/data/winter/20250207/diffs/WINTERca...,2025-02-08T11:08:32.515195+00:00,0.19.0
84,2023A001,2025-02-08,S250206dm,34.643055,52.341393,2,2025-02-08T15:02:14.115000+00:00,4268,diff,/data/loki/data/winter/20250207/diffs/WINTERca...,2025-02-08T11:08:32.425370+00:00,0.19.0


In [10]:
for time in list(set(images["utctime"]))[:1]:
    df = images[images["utctime"] == time]
    print(df)

    progname   nightdate   targname        ra        dec  fid  \
43  2023A001  2025-02-08  S250206dm  37.98403  54.060917    2   

                             utctime  fieldid image_type  \
43  2025-02-08T12:07:59.626000+00:00     3842       diff   

                                             savepath  \
43  /data/loki/data/winter/20250207/diffs/WINTERca...   

                        lastmodified pipeversion  
43  2025-02-08T11:08:25.113954+00:00      0.19.0  


In [11]:
row = images.iloc[0]
row

progname                                                 2023A001
nightdate                                              2025-02-08
targname                                                S250206dm
ra                                                      36.099724
dec                                                      55.32487
fid                                                             2
utctime                          2025-02-08T15:54:14.173000+00:00
fieldid                                                      3635
image_type                                                   diff
savepath        /data/loki/data/winter/20250207/diffs/WINTERca...
lastmodified                     2025-02-08T11:08:34.697189+00:00
pipeversion                                                0.19.0
Name: 0, dtype: object

In [12]:
# mocs = MOC.from_boxes(
#     lon=images["ra"].to_numpy() * u.deg,
#     lat=images["dec"].to_numpy()  * u.deg,
#     a=0.5 * u.deg,
#     b=0.4 * u.deg,
#     angle=0 * u.deg,
#     max_depth=hp.nside2order(nu.nside),  # Match winter nside/order to skymap nside/order
# )

mocs = MOC.from_boxes(
    lon=images["ra"].to_numpy() * u.deg,
    lat=images["dec"].to_numpy()  * u.deg,
    a=0.25 * u.deg,
    b=0.2 * u.deg,
    angle=0 * u.deg,
    max_depth=hp.nside2order(nu.nside),  # Match winter nside/order to skymap nside/order
)

In [13]:
# images["ra"]

In [14]:
print(nu.nside)

1024


In [15]:
pixels = [x.flatten() for x in mocs]
all_times = [Time(x.split("+")[0], format="isot") for x in images["utctime"]]

In [16]:
pix_obs_times = dict()

for i, obs_time in tqdm(enumerate(all_times), total=len(all_times)):
    # obs = data[data["obsjd"] == obs_time]
    # flatpix = mocs[i].flatten()
    # print(flatpix)
    
    pixels = hp.nest2ring(nu.nside, np.array(mocs[i].flatten(), dtype=int))
    print(pixels)

    for p in pixels:
        if p not in pix_obs_times.keys():
            pix_obs_times[p] = [obs_time]
        else:
            pix_obs_times[p] += [obs_time]

100%|████████████████████████████████████████| 86/86 [00:00<00:00, 11471.51it/s]

[1132816 1132812 1132814 1132815 1129807 1129806 1126802 1132813 1129805
 1129804 1126800 1126801 1123801 1123800 1120804 1132811 1129803 1129802
 1126798 1123797 1126799 1123799 1123798 1120802 1120803 1117811 1117810
 1114822 1120801 1117809 1114820 1114821 1111837 1111836 1108856 1108855
 1105879 1129808 1126804 1126803 1123803 1123802 1120806 1120807 1117814
 1114826 1120805 1117813 1117812 1114824 1114825 1111841 1111840 1108860
 1111842 1108861 1105885 1105884 1102912 1099944 1114823 1111839 1111838
 1108858 1108859 1105883 1105882 1102910 1108857 1105881 1105880 1102908
 1102909 1099941 1099940 1102911 1099943 1099942 1102907 1099939 1099938]
[1153987 1153986 1150950 1153985 1153984 1150948 1150949 1147917 1147916
 1144888 1150951 1147919 1147918 1144890 1141866 1144889 1141865 1141864
 1138844 1138845 1135829 1135828 1132816 1153983 1153982 1150946 1150945
 1147913 1144884 1150947 1147915 1147914 1144886 1144887 1141863 1141862
 1138842 1144885 1141861 1141860 1138840 1138841 1

In [ ]:
min_sep=0.01

npix = hp.nside2npix(nu.nside)

ras, decs = hp.pixelfunc.pix2ang(
    nu.nside, hp.nest2ring(nu.nside, nu.pixel_nos), lonlat=True
)

radecs = SkyCoord(ra=ras * u.deg, dec=decs * u.deg)

radecs

coverage_data = []

times = []

for i, p in enumerate(tqdm(hp.nest2ring(nu.nside, nu.pixel_nos))):
    entry = {
        "pixel_no": p,
        "prob": nu.map_probs[i],
        "gal_b": radecs[i].galactic.b.deg,
        "in_plane": abs(radecs[i].galactic.b.deg) < 10.0,
        "ra_deg": ras[i],
        "dec_deg": decs[i],
    }

    if p in pix_obs_times.keys():
        obs = pix_obs_times[p]

        entry["n_det_class"] = 2 if ((max(obs) - min(obs)) > min_sep) else 1

        entry["latency_days"] = min(obs) - nu.t_min.jd

        # overlapping_fields += pix_map[p]

        times += list(obs)
    else:
        entry["n_det_class"] = 0
        entry["latency_days"] = np.nan

    coverage_data.append(entry)

coverage_df = pd.DataFrame(coverage_data)

# _observations = data.query("obsjd in @times").reset_index(drop=True)[
#     ["obsjd", "exposure_time", "filter_id"]
# ]
# bands = [self.fid_to_band(fid) for fid in _observations["filter_id"].values]
# _observations["band"] = bands
# _observations.drop(columns=["filter_id"], inplace=True)
# self.observations = _observations

# self.logger.info("All observations:")
# self.logger.info(f"\n{self.observations}")

try:
    first_obs = Time(min(times), format="jd")
    first_obs.utc.format = "isot"
    last_obs = Time(max(times), format="jd")
    last_obs.utc.format = "isot"

except ValueError:
    err = (
        f"No observations of this event were found at any time between "
        f"{nu.t_min} and {nu.t_min + first_det_window_days * u.day}. "
        f"Coverage overlap is 0%!"
    )
    raise ValueError(err)

# self.logger.info(f"Observations started at {self.first_obs.isot}")

# return (
#     coverage_df,
#     times,
#     overlapping_fields,
# )

  3%|█                                   | 8265/265408 [00:09<04:34, 938.20it/s]WARNING: TimeDeltaMissingUnitWarning: Numerical value without unit or explicit format passed to TimeDelta, assuming days [astropy.time.core]
/Users/robertstein/anaconda3/envs/nuwinter_env/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
 41%|█████████████▉                    | 108882/265408 [02:00<02:52, 904.90it/s]

In [ ]:
first_obs

In [ ]:
coverage_df

In [ ]:
import matplotlib.patches as mpatches

fig = plt.figure()
plt.subplot(projection="aitoff")

overlap_mask = (coverage_df["n_det_class"] == 2) & ~coverage_df["in_plane"]
overlap_prob = coverage_df[overlap_mask]["prob"].sum() * 100.0

size = hp.max_pixrad(nu.nside) ** 2 * 50.0

veto_pixels = coverage_df.where(coverage_df["n_det_class"] == 0)

if len(veto_pixels) > 0:
    plt.scatter(
        np.radians(nu.wrap_around_180(veto_pixels["ra_deg"])),
        np.radians(veto_pixels["dec_deg"]),
        color="red",
        s=size,
    )

plane_pixels = coverage_df.where(
    coverage_df["in_plane"] & (coverage_df["n_det_class"] > 0)
)

if len(plane_pixels) > 0:
    plt.scatter(
        np.radians(nu.wrap_around_180(plane_pixels["ra_deg"])),
        np.radians(plane_pixels["dec_deg"]),
        color="green",
        s=size,
    )

single_pixels = coverage_df.where(
    ~coverage_df["in_plane"] & (coverage_df["n_det_class"] == 1)
)

if len(single_pixels) > 0:
    plt.scatter(
        np.radians(nu.wrap_around_180(single_pixels["ra_deg"])),
        np.radians(single_pixels["dec_deg"]),
        c=single_pixels["prob"],
        vmin=0.0,
        vmax=max(nu.data[nu.key]),
        s=size,
        cmap="gray",
    )

double_pixels = coverage_df.where(
    ~coverage_df["in_plane"] & (coverage_df["n_det_class"] == 2)
)

if len(double_pixels) > 0:
    plt.scatter(
        np.radians(nu.wrap_around_180(double_pixels["ra_deg"])),
        np.radians(double_pixels["dec_deg"]),
        c=double_pixels["prob"],
        vmin=0.0,
        vmax=max(nu.data[nu.key]),
        s=size,
    )

red_patch = mpatches.Patch(color="red", label="Not observed")
gray_patch = mpatches.Patch(color="gray", label="Observed once")
violet_patch = mpatches.Patch(
    color="green", label="Observed Galactic Plane (|b|<10)"
)
plt.legend(handles=[red_patch, gray_patch, violet_patch])

message = (
    "In total, {0:.1f} % of the contour was observed at least once.\n"
    "This estimate includes {1:.1f} % of the contour "
    "at a galactic latitude <10 deg.\n"
    "In total, {2:.1f} % of the contour was observed at least twice. \n"
    "In total, {3:.1f} % of the contour was observed at least twice, "
    "and excluding low galactic latitudes.\n"
    "These estimates account for chip gaps.".format(
        100.0 * coverage_df.query("n_det_class > 0")["prob"].sum(),
        100.0 * coverage_df.query("in_plane & n_det_class > 0")["prob"].sum(),
        100.0 * coverage_df.query("n_det_class == 2")["prob"].sum(),
        100.0 * coverage_df.query("n_det_class == 2 & ~in_plane")["prob"].sum(),
    )
)

n_pixels = len(coverage_df.query("n_det_class > 0 & prob > 0.0"))
n_double = len(coverage_df.query("n_det_class == 2 & prob > 0.0"))
n_plane = len(coverage_df.query("in_plane & n_det_class > 0 & prob > 0.0"))

healpix_area = nu.pixel_area * n_pixels
double_extragalactic_area = nu.pixel_area * n_double
plane_area = nu.pixel_area * n_plane

# overlap_fields = overlapping_fields

logger.info(
    f"{n_pixels} pixels were covered, covering approximately "
    f"{healpix_area:.2g} sq deg."
)
logger.info(
    f"{n_double} pixels were covered at least twice (b>10), "
    f"covering approximately {double_extragalactic_area:.2g} sq deg."
)
logger.info(
    f"{n_plane} pixels were covered at low galactic latitude, "
    f"covering approximately {plane_area:.2g} sq deg."
)

print(message)

In [ ]:
overlap_prob